# Parsing the datasets

### Importing libraries

In [346]:
# Numpy
import numpy as np
from numpy import concatenate, array
from numpy.random import randn
# Decimal precision value to display in the matrix
np.set_printoptions(precision=5, suppress=True)

# Scipy
import scipy
import scipy.stats as stats

# Matplotlib
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
#mpl.rc('figure', figsize=(10, 8))

# DBscan from sklearn
from sklearn import cluster, datasets
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

# Pandas experiments
import pandas as pd
from pandas import Series, DataFrame, Panel

# Misc
import time
import datetime as dt
import math
import random
print 'All libraries loaded.'

All libraries loaded.


# Make dataframes from data

In [134]:
# Make dataframes from data

# eGFR data
egfr_df = pd.read_csv('~/code/independent/datasets/cdr_gfr_derived.csv', parse_dates=['resultdata'])
egfr_df.drop('gfr', axis=1, inplace=True)
egfr_df.columns = ['pid', 'timestamp', 'gender', 'birthyear', 'age', 'gfr']

# Findings data
findings_df = pd.read_csv('~/code/independent/datasets/cdr_finding.csv', parse_dates=['finddate'], usecols=['idperson', 'finddate', 'valuename', 'findvalnum'])
findings_df = findings_df[['idperson', 'finddate', 'valuename', 'findvalnum']]
findings_df.columns = ['pid', 'timestamp', 'testname', 'testval']

# Lab reports data
lab_df = pd.read_csv('~/code/independent/datasets/cdr_lab_result.csv', parse_dates=['resultdate'], usecols=['idperson', 'resultdate', 'valuename', 'resultvaluenum'])
lab_df = lab_df[['idperson', 'resultdate', 'valuename', 'resultvaluenum']]
lab_df.columns = ['pid', 'timestamp', 'testname', 'testval']
# Make all lab tests values uppercase
lab_df.testname = map(lambda x: x.upper(), lab_df.testname)

In [135]:
# Normalize dates (to remove the time part of it)
egfr_df.timestamp = egfr_df.timestamp.map(pd.datetools.normalize_date)
findings_df.timestamp = findings_df.timestamp.map(pd.datetools.normalize_date)
lab_df.timestamp = lab_df.timestamp.map(pd.datetools.normalize_date)

In [136]:
# NaN values
total_rowcount = len(lab_df.testval.values)
nan_rowcount = len([x for x in lab_df.testval.values if math.isnan(x)])
print '\n',(str(nan_rowcount*100/total_rowcount)+ "% of the values are NaN")

# Drop NaN rows
# lab_df.dropna(inplace=True)


1% of the values are NaN


## Make tiny versions of the datasets

In [137]:
# howmany = 500000

# egfr_df = egfr_df[:howmany]
# lab_df = lab_df[:howmany]
# findings_df = findings_df[:howmany]

## Make a multi-index/hierarchical index 
#### The index will be a combination of the PID (person ID) and timestamp

In [140]:
# Set the index as a combination of the person ID and timestamp
egfr_df.set_index(['pid', 'timestamp'], inplace=True)
findings_df.set_index(['pid', 'timestamp'], inplace=True)
lab_df.set_index(['pid', 'timestamp'], inplace=True)

## Get list of all patients for whom we have eGFR data

In [340]:
list_of_patients = list(set(egfr_df.index.get_level_values('pid').values))
print '\nFound', len(list_of_patients), 'unique patients'


Found 63215 unique patients


## Get small sample of patients

In [420]:
# Get just 5 patients
# list_of_patients = random.sample(list_of_patients, 5)

# list_of_patients = [8555317, 8555928, 8565179]

## Make the new dataframe (empty)

In [343]:
# Column names will be a combination of all the lab test names and finding names
unique_findings = set(findings_df.testname.values)
unique_labtests = set(lab_df.testname.values)
print '\nFindings:', list(unique_findings)
print 'Lab tests:', list(unique_labtests)
final_col_names = np.append(list(unique_findings), list(unique_labtests))


Findings: ['FND_BPS', 'FND_BPD']
Lab tests: ['LR_AST', 'LR_MICROCR', 'LR_HDL', 'LR_TRIG', 'LR_A1C', 'LR_CR', 'LR_PTH', 'LR_GLUCNONFAST', 'LR_LDL', 'LR_GFR', 'LR_VITD 25', 'LR_ALT', 'LR_PHOS', 'LR_GFR_AFRAMER']


In [366]:
# Row locations in the eGFR dataframe of patients
# locations_of_patients = []
# for patient in list_of_patients:
#     get_loc_result = egfr_df.index.get_loc(patient)
#     locations_of_patients += [i for i,v in enumerate(get_loc_result) if v == True]

# combined_df = DataFrame(egfr_df.iloc[locations_of_patients])
combined_df = pd.DataFrame(egfr_df)

# Add the new columns
for newcol in final_col_names:
    combined_df[newcol] = np.nan

## Fill values into the new dataframe<br>based on joins on the other dataframes

In [414]:
def fill_data(source_df, patient_id):
    
    ############### Could optimize
    locations_of_patients = []
    if str(patient_id) in source_df.index:
        
#         get_loc_result = source_df.index.get_loc(str(patient_id))
#         locations_of_patients += [i for i,v in enumerate(get_loc_result) if v == True]
#         this_patients_data = source_df.iloc[locations_of_patients]
        this_patients_data = source_df.loc[str(patient_id)]

        # We only need data for the dates that exist in the combined DF already 
        # (the ones that have eGFR readings)
        for timestamp in combined_df.loc[patient_id].index:
            if timestamp in this_patients_data.index.get_level_values('timestamp'):
                this_patient = this_patients_data.loc[timestamp]

                # Set the value in the DF for this test
                # Check if there's multiple values for the same timestamp
                if type(this_patient) == pd.core.frame.DataFrame:
                    for i in range(this_patient.testname.size):
                        
                        try:
                            combined_df.loc[(patient_id, timestamp), this_patient.testname[i]] = this_patient.testval[i]
                        except:
                            print 'multivalues for', patient_id
                else:
                    combined_df.loc[(patient_id, timestamp), this_patient.testname] = this_patient.testval

                    
fill_count = 0
start_time = time.time()
lastseen_time = time.time()

for patient_id in list_of_patients:
    
    if fill_count % 500 == 0:
        elapsed_time = time.time() - lastseen_time
        lastseen_time = time.time()
        print 'Data fill for', fill_count, 'patients completed in', '{:.3f}'.format(elapsed_time), 'seconds'
    fill_count += 1
    
#     ########################################## REMOVE
#     if fill_count < 14001:
#         continue
    
    # Check if the findings df has data about this patient
    fill_data(findings_df, patient_id)

    # Check if the lab-tests df has data about this patient
    fill_data(lab_df, patient_id)
    
    
elapsed_time = time.time() - start_time
print 'Task completed in', '{:.3f}'.format(elapsed_time), 'seconds'

Data fill for 0 patients completed in 0.000 seconds
Data fill for 500 patients completed in 0.014 seconds
Data fill for 1000 patients completed in 0.014 seconds
Data fill for 1500 patients completed in 0.014 seconds
Data fill for 2000 patients completed in 0.014 seconds
Data fill for 2500 patients completed in 0.017 seconds
Data fill for 3000 patients completed in 0.019 seconds
Data fill for 3500 patients completed in 0.014 seconds
Data fill for 4000 patients completed in 0.013 seconds
Data fill for 4500 patients completed in 0.022 seconds
Data fill for 5000 patients completed in 0.015 seconds
Data fill for 5500 patients completed in 0.013 seconds
Data fill for 6000 patients completed in 0.013 seconds
Data fill for 6500 patients completed in 0.013 seconds
Data fill for 7000 patients completed in 0.012 seconds
Data fill for 7500 patients completed in 0.013 seconds
Data fill for 8000 patients completed in 0.012 seconds
Data fill for 8500 patients completed in 0.011 seconds
Data fill for 

# Write the resultant DF to a CSV file

In [408]:
# Write the code for writing to CSV

In [419]:
# How many values are NaNs
total_rowcount = len(combined_df.FND_BPS.values)
nan_rowcount = len([x for x in combined_df.FND_BPS.values if math.isnan(x)])
print '\n',(str(nan_rowcount*100/total_rowcount)+ "% of the values are NaN")
combined_df.size


86% of the values are NaN


9322160

## Write the dataframe to a CSV

In [421]:
output_file_path = 'combined_df.csv'
combined_df.to_csv(output_file_path, sep='\t')

In [423]:
egfr_df.shape

(466108, 20)

In [422]:
combined_df.shape

(466108, 20)

In [ ]:
# lab_df.loc['8555580', '2010-02-22']

# lab_df.loc['8554722'].loc['2009-02-20']
# lab_df.loc['8554722']

lab_df.iloc[lab_df.index.get_level_values('pid')=='8554722']

# x = lab_df.xs('8555580', level='pid')['2010-02-22']
# x

print lab_df.xs('8555580', level='pid')['2010-02-22']

# lab_df
# lab_df['8554722']['2008-04-07']
# lab_df['2010-02-22']

# .xs('2009-02-20', level='timestamp')
# x.index.searchsorted(stamp)
# x['2008-02-04']